In [1]:
import os
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# Scopes allow us to read and write tasks
SCOPES = ['https://www.googleapis.com/auth/tasks']

# Authentication
def authenticate_google_tasks():
    creds = None
    # The token.pickle stores the user's access and refresh tokens
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or expired, get new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('tasks', 'v1', credentials=creds)

# Create a new task list
def create_task_list(service, title):
    tasklist = {
        'title': title
    }
    result = service.tasklists().insert(body=tasklist).execute()
    print(f"Task list created: {result['title']}, ID: {result['id']}")
    return result['id']

# Get all task lists
def get_task_lists(service):
    result = service.tasklists().list().execute()
    tasklists = result.get('items', [])
    if not tasklists:
        print("No task lists found.")
    else:
        for tasklist in tasklists:
            print(f"Task List: {tasklist['title']}, ID: {tasklist['id']}")
    return tasklists

# Create a new task in a specific task list
def create_task_in_list(service, tasklist_id, title, due_date=None):
    task = {
        'title': title,
    }
    if due_date:
        task['due'] = due_date
    result = service.tasks().insert(tasklist=tasklist_id, body=task).execute()
    print(f"Task created in list: {result['title']}")

# Get tasks from a specific task list
def get_tasks_from_list(service, tasklist_id):
    results = service.tasks().list(tasklist=tasklist_id).execute()
    tasks = results.get('items', [])
    if not tasks:
        print("No tasks found.")
    else:
        for task in tasks:
            print(f"Task: {task['title']}, Due: {task.get('due', 'No due date')}")

# Delete a task by task ID
def delete_task(service, tasklist_id, task_id):
    service.tasks().delete(tasklist=tasklist_id, task=task_id).execute()
    print("Task deleted.")

# Main execution with an example of creating and managing tasks
if __name__ == '__main__':
    service = authenticate_google_tasks()

    # Create a new task list called "Schule"
    schule_tasklist_id = "T2tteVU3OWdwVnQ2Q3pfNw"
 
    # Create a new task list called "Schule" if it doesn't exist
    create_task_in_list(service, schule_tasklist_id, "Study for exam", "2024-10-15T12:00:00.000Z")
    # Get tasks from "Schule" task list
    print("\nTasks in 'Schule' task list:")
    get_tasks_from_list(service, schule_tasklist_id)



Task created in list: Study for exam

Tasks in 'Schule' task list:
Task: Study for exam, Due: 2024-10-15T00:00:00.000Z
Task: Finish Homework, Due: 2024-10-06T00:00:00.000Z
Task: Passed: testtest, Due: 2024-10-03T00:00:00.000Z
Task: Passed: test, Due: 2024-10-03T00:00:00.000Z
Task: Passed: hiiiiii, Due: 2024-10-03T00:00:00.000Z


In [2]:
print("\nTasks in 'Schule' task list:")
get_tasks_from_list(service, schule_tasklist_id)


Tasks in 'Schule' task list:
Task: Study for exam, Due: 2024-10-15T00:00:00.000Z
Task: Finish Homework, Due: 2024-10-06T00:00:00.000Z
Task: Passed: testtest, Due: 2024-10-03T00:00:00.000Z
Task: Passed: test, Due: 2024-10-03T00:00:00.000Z
Task: Passed: hiiiiii, Due: 2024-10-03T00:00:00.000Z


In [3]:
from datetime import datetime, timedelta, timezone
import time

# Flag tasks as "Passed" after their due date, and mark them as "Completed" one day later
def process_tasks(service, tasklist_id):
    results = service.tasks().list(tasklist=tasklist_id).execute()
    tasks = results.get('items', [])
    current_time = datetime.now(timezone.utc).date()  # Use timezone-aware datetime object, but only compare the date

    for task in tasks:
        # Check if the task has a due date
        if 'due' in task:
            # Convert the task due date to a timezone-aware datetime object
            task_due_date = datetime.strptime(task['due'], "%Y-%m-%dT%H:%M:%S.%fZ").date()
            
            # Check if the task's due date has passed
            if task_due_date < current_time:
                # Add the "Passed" flag if it hasn't already been added
                if not task['title'].startswith('Passed'):
                    task['title'] = f"Passed: {task['title']}"
                    service.tasks().update(tasklist=tasklist_id, task=task['id'], body=task).execute()
                
                # Check if more than a day has passed since the due date and mark it as completed
                if current_time > task_due_date + timedelta(days=1) and task['status'] != 'completed':
                    task['status'] = 'completed'
                    service.tasks().update(tasklist=tasklist_id, task=task['id'], body=task).execute()

# Main loop to automate the process (check every second)
def monitor_tasks(service, tasklist_id):
    while True:
        process_tasks(service, tasklist_id)
        time.sleep(1)  # Check every second

# Example usage:
if __name__ == '__main__':
    service = authenticate_google_tasks()

    # Assuming you want to monitor the "Schule" task list
    schule_tasklist_id = "T2tteVU3OWdwVnQ2Q3pfNw"

    monitor_tasks(service, schule_tasklist_id)


KeyboardInterrupt: 

In [9]:
from datetime import datetime, timezone
import time
import os
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# Scopes allow us to read and write tasks
SCOPES = ['https://www.googleapis.com/auth/tasks']

# Authentication
def authenticate_google_tasks():
    creds = None
    # The token.pickle stores the user's access and refresh tokens
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or expired, get new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('tasks', 'v1', credentials=creds)

# Get Task List ID by Task List Title
def get_tasklist_id_by_title(service, title):
    result = service.tasklists().list().execute()
    tasklists = result.get('items', [])
    for tasklist in tasklists:
        if tasklist['title'].lower() == title.lower():
            return tasklist['id']
    raise ValueError(f"Tasklist '{title}' not found")

# Get Task ID by Task Title in a specific Task List
def get_task_id_by_title(service, tasklist_id, title):
    result = service.tasks().list(tasklist=tasklist_id).execute()
    tasks = result.get('items', [])
    for task in tasks:
        if task['title'].lower() == title.lower():
            return task['id']
    raise ValueError(f"Task '{title}' not found in task list with ID '{tasklist_id}'")

# Create a Task
def create_task(service, tasklist_id, title, due_date=None, priority=None, description=None):
    task_body = {
        'title': title
    }
    # Add optional parameters if provided
    if due_date:
        task_body['due'] = due_date
    if priority:
        task_body['notes'] = f"Priority: {priority}"  # Notes used for priority in Google Tasks
    if description:
        task_body['notes'] = (task_body['notes'] + f"\nDescription: {description}") if 'notes' in task_body else f"Description: {description}"

    task = service.tasks().insert(tasklist=tasklist_id, body=task_body).execute()
    print(f"Created task '{title}' with ID: {task['id']}")
    return task['id']

# Update a Task (optional params: title, due date, priority, description)
def update_task(service, tasklist_id, task_id, title=None, due_date=None, priority=None, description=None):
    task = service.tasks().get(tasklist=tasklist_id, task=task_id).execute()

    # Update fields only if new values are provided
    if title:
        task['title'] = title
    if due_date:
        task['due'] = due_date
    if priority or description:
        notes = task.get('notes', "")
        if priority:
            notes = f"Priority: {priority}"
        if description:
            notes += f"\nDescription: {description}"
        task['notes'] = notes

    updated_task = service.tasks().update(tasklist=tasklist_id, task=task_id, body=task).execute()
    print(f"Updated task '{task_id}'")
    return updated_task

# Get all tasks from a specific task list
def get_tasks(service, tasklist_id):
    result = service.tasks().list(tasklist=tasklist_id).execute()
    print(result)
    tasks = result.get('items', [])
    if not tasks:
        print(f"No tasks found in task list with ID '{tasklist_id}'")
        return []
    for task in tasks:
        print(f"Task: {task['title']} | Due Date: {task.get('due', 'No due date')} | Notes: {task.get('notes', 'No notes')}")
    return tasks

# Example usage:
if __name__ == '__main__':
    service = authenticate_google_tasks()

    # Example: Get task list ID by title (you can replace "Schule" with any list name)
    tasklist_id = get_tasklist_id_by_title(service, "Schule")

    # Example: Update a task by getting the task ID from the task title
    # Example: Get all tasks
    get_tasks(service, tasklist_id)


{'kind': 'tasks#tasks', 'etag': '"MTQ5OTA1ODc0OQ"', 'items': [{'kind': 'tasks#task', 'id': 'SHYwZnJTVXFKdFBSM1Q5UQ', 'etag': '"MTQ5OTA2MTMwMg"', 'title': 'aaaa', 'updated': '2024-10-04T21:05:14.000Z', 'selfLink': 'https://www.googleapis.com/tasks/v1/lists/bDBRLVVrSjZxTE5fMDJuRQ/tasks/SHYwZnJTVXFKdFBSM1Q5UQ', 'position': '00000000000000000002', 'status': 'needsAction', 'due': '2024-10-24T00:00:00.000Z', 'links': [], 'webViewLink': 'https://tasks.google.com/task/Hv0frSUqJtPR3T9Q'}, {'kind': 'tasks#task', 'id': 'QjFfSGhQY0xVQlV3WnZQUQ', 'etag': '"MTQ5OTA1NDU5MA"', 'title': 'fsdfsdf', 'updated': '2024-10-04T21:05:07.000Z', 'selfLink': 'https://www.googleapis.com/tasks/v1/lists/bDBRLVVrSjZxTE5fMDJuRQ/tasks/QjFfSGhQY0xVQlV3WnZQUQ', 'position': '00000000000000000001', 'status': 'needsAction', 'links': [], 'webViewLink': 'https://tasks.google.com/task/B1_HhPcLUBUwZvPQ'}, {'kind': 'tasks#task', 'id': 'YlpmMXN3ZHBnSVRNQXFLeg', 'etag': '"MTQ5OTA1MjMxNw"', 'title': 'afasfs', 'updated': '2024-10-04

In [1]:
from datetime import datetime, timezone
from typing import Optional, Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
import time
import os
import pickle
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import dateutil.parser

# Scopes allow us to read and write tasks
SCOPES = ['https://www.googleapis.com/auth/tasks']

# Authentication
def authenticate_google_tasks():
    creds = None
    # The token.pickle stores the user's access and refresh tokens
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If credentials are not available or expired, get new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('tasks', 'v1', credentials=creds)

# Get Task List ID by Task List Title
def get_tasklist_id_by_title(service, title):
    result = service.tasklists().list().execute()
    tasklists = result.get('items', [])
    for tasklist in tasklists:
        if tasklist['title'].lower() == title.lower():
            return tasklist['id']
    raise ValueError(f"Tasklist '{title}' not found")

# Define the input schema for creating a task
class CreateTaskInput(BaseModel):
    task_title: str = Field(description="Title of the task to create")
    due_date: Optional[str] = Field(default=None, description="Due date in RFC3339 or other common date formats, None if no due date")
    priority: Optional[str] = Field(default=None, description="Priority level of the task")
    description: Optional[str] = Field(default=None, description="Description of the task")

# Define the custom tool for creating a task in Google Tasks
class CreateTaskTool(BaseTool):
    name: str = "create_task"
    description: str = "Tool for creating a new task in Google Tasks."
    args_schema: Type[BaseModel] = CreateTaskInput
    return_direct: bool = False

    def _run(
        self, task_title: str, due_date: Optional[str] = None, priority: Optional[str] = None, description: Optional[str] = None, run_manager: Optional = None
    ) -> str:
        """Create a new task in Google Tasks."""
        # Authenticate and get the Google Tasks service
        service = authenticate_google_tasks()

        # Get the task list ID
        tasklist_id = get_tasklist_id_by_title(service, "Schule")

        # Parse the due date if provided
        parsed_due_date = None
        if due_date:
            try:
                parsed_due_date = datetime.fromisoformat(due_date).isoformat()
            except ValueError:
                try:
                    parsed_due_date = dateutil.parser.parse(due_date).isoformat()
                except ValueError:
                    raise ValueError("Invalid due date format. Please provide a valid date.")

        # Prepare the task body
        task_body = {'title': task_title}
        if parsed_due_date:
            task_body['due'] = parsed_due_date
        if priority:
            task_body['notes'] = f"Priority: {priority}"
        if description:
            task_body['notes'] = (task_body['notes'] + f"\nDescription: {description}") if 'notes' in task_body else f"Description: {description}"

        # Create the task
        task = service.tasks().insert(tasklist=tasklist_id, body=task_body).execute()
        return f"Created task '{task_title}' with ID: {task['id']}"

# Define the input schema for getting the current date
class GetCurrentDateInput(BaseModel):
    format: Optional[str] = Field(default="RFC3339", description="Format in which to return the current date (e.g., RFC3339)")

# Define the custom tool for getting the current date
class GetCurrentDateTool(BaseTool):
    name: str = "get_current_date"
    description: str = "Tool for retrieving the current date in the specified format."
    args_schema: Type[BaseModel] = GetCurrentDateInput
    return_direct: bool = False

    def _run(self, format: Optional[str] = "RFC3339", run_manager: Optional = None) -> str:
        """Get the current date in the specified format."""
        current_date = datetime.now(timezone.utc)
        if format == "RFC3339":
            return current_date.isoformat()
        else:
            return current_date.strftime(format)

# Instantiate the tools
create_task_tool = CreateTaskTool()
get_current_date_tool = GetCurrentDateTool()

In [2]:
import discord
import nest_asyncio
import asyncio
from dotenv import load_dotenv
import os
import base64
from flask import Flask, request, jsonify
import threading
from PIL import Image
import io

# Import ChatBot dependencies
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

load_dotenv()

# Replace with your GitHub token
azure_token = os.getenv("azure_token")

# Mycroft agent configuration
model_name = "gpt-4o-mini"
endpoint = "https://models.inference.ai.azure.com"
system_prompt = """You are an agent designed to assist students with their homework by creating Google Tasks.
Students will write their homework assignments to you, and you will process these assignments and create corresponding tasks in Google Tasks.
Ensure that each task is clearly defined and includes all necessary details provided by the student.
If a due date is mentioned in relative terms (e.g., "next week", "tomorrow"), use the current date to determine the exact due date in RFC3339 format.
If you encounter any issues or errors while creating the tasks, debug your process and try again.
If it does not seem like you can create a task based on the provided information, just return "I don't know" as the answer.
Always use the same language as the user asking.
"""

tools = [get_current_date_tool, create_task_tool]
# Initialize the language model
llm = ChatOpenAI(
    model_name=model_name,
    base_url=endpoint,
    api_key=azure_token
)
# Initialize memory saver
memory = MemorySaver()

# Create the agent executor
agent_executor = create_react_agent(
    llm, tools, checkpointer=memory, state_modifier=system_prompt
)

config = {"configurable": {"thread_id": "default", "recursion_limit": 1000}}

# Example function to send a message to the agent
def agent_send_message(message):
    human_message = HumanMessage(content=message)
    response = agent_executor.invoke(
        {"messages": [human_message]},
        config=config,
    )
    return response

def get_most_recent_ai_message_content_and_tool_calls(response):
    messages = response.get('messages', [])
    most_recent_content = None
    tool_calls = []

    for message in reversed(messages):
        if isinstance(message, HumanMessage):
            break
        if isinstance(message, AIMessage):
            if message.content:
                most_recent_content = message.content
            if 'tool_calls' in message.additional_kwargs:
                tool_calls.extend(message.additional_kwargs.get('tool_calls', []))

    return most_recent_content, tool_calls

In [3]:
response = agent_send_message("mathe klasur am übernächsten montag")
print(get_most_recent_ai_message_content_and_tool_calls(response))

('Die Aufgabe "Mathe Klausur" wurde erfolgreich erstellt. Sie ist fällig am 14. Oktober 2024 und beinhaltet die Vorbereitung auf die Mathe Klausur.', [{'id': 'call_u2ITepqovZbfMJ0F16vvJqus', 'function': {'arguments': '{"task_title":"Mathe Klausur","due_date":"2024-10-14T00:00:00Z","description":"Vorbereitung auf die Mathe Klausur am übernächsten Montag."}', 'name': 'create_task'}, 'type': 'function'}, {'id': 'call_IYgLs8Be3CAftZdrWI2qfMxk', 'function': {'arguments': '{"format":"RFC3339"}', 'name': 'get_current_date'}, 'type': 'function'}])


In [ ]:
import discord
import nest_asyncio
import asyncio
import os
from dotenv import load_dotenv
from discord.ext import tasks
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

load_dotenv()

TOKEN = os.getenv('DISCORD_TOKEN')
CHANNEL_NAME = 'hausaufgaben'
azure_token = os.getenv("AZURE_TOKEN")

# Mycroft agent configuration
model_name = "gpt-4o-mini"
endpoint = "https://models.inference.ai.azure.com"
system_prompt = """You are an agent designed to assist students with their homework by creating Google Tasks.
Students will write their homework assignments to you, and you will process these assignments and create corresponding tasks in Google Tasks.
Ensure that each task is clearly defined and includes all necessary details provided by the student.
If a due date is mentioned in relative terms (e.g., "next week", "tomorrow"), use the current date to determine the exact due date in RFC3339 format.
If you encounter any issues or errors while creating the tasks, debug your process and try again.
If it does not seem like you can create a task based on the provided information, just return "I don't know" as the answer.
Always use the same language as the user asking.
"""

# Initialize the language model
llm = ChatOpenAI(
    model_name=model_name,
    base_url=endpoint,
    api_key=azure_token
)

# Initialize memory saver
memory = MemorySaver()

# Create the agent executor
tools = [get_current_date_tool, create_task_tool]
agent_executor = create_react_agent(
    llm, tools, checkpointer=memory, state_modifier=system_prompt
)

# Initialize the Discord Bot
intents = discord.Intents.default()
intents.messages = True
intents.guilds = True
intents.message_content = True

bot = discord.Client(intents=intents)
service = authenticate_google_tasks()
tasklist_id = None
pinned_message_id = None  # Store the pinned message ID to update it

# Fetch all tasks from the tasklist
def get_tasks(service, tasklist_id):
    result = service.tasks().list(tasklist=tasklist_id).execute()
    return result.get('items', [])

# Fetch pending tasks (tasks that are not completed)
def get_pending_tasks(service, tasklist_id):
    tasks = get_tasks(service, tasklist_id)
    pending_tasks = []
    now = datetime.now(timezone.utc)

    for task in tasks:
        due_date = task.get('due')
        if task.get('status') != 'completed':  # Only display tasks that are not completed
            if due_date:
                due_datetime = datetime.fromisoformat(due_date[:-1] + '+00:00')
                if due_datetime > now:
                    pending_tasks.append({
                        'title': task['title'],
                        'due_date': due_datetime.strftime('%Y-%m-%d')
                    })
            else:
                # Include tasks without a due date in pending tasks
                pending_tasks.append({
                    'title': task['title'],
                    'due_date': 'No due date'
                })

    return pending_tasks

# Fetch passed tasks (tasks where the due date has passed and they are not marked as completed)
def get_passed_tasks(service, tasklist_id):
    tasks = get_tasks(service, tasklist_id)
    passed_tasks = []
    now = datetime.now(timezone.utc)

    for task in tasks:
        due_date = task.get('due')
        if task.get('status') != 'completed':  # Only consider non-completed tasks
            if due_date:
                due_datetime = datetime.fromisoformat(due_date[:-1] + '+00:00')
                if due_datetime < now:
                    passed_tasks.append({
                        'title': task['title'],
                        'due_date': due_datetime.strftime('%Y-%m-%d')
                    })

    return passed_tasks

# Example to display the tasks in a pinned message
def display_tasks(service, tasklist_id):
    pending_tasks = get_pending_tasks(service, tasklist_id)
    passed_tasks = get_passed_tasks(service, tasklist_id)

    pending_tasks_md = '\n'.join([f"- **{task['title']}** (Due: {task['due_date']})" for task in pending_tasks])
    passed_tasks_md = '\n'.join([f"- __{task['title']}__ (Was due: {task['due_date']})" for task in passed_tasks])

    message = (
        "### Tasks Overview\n"
        f"**Pending Tasks:**\n{pending_tasks_md if pending_tasks_md else 'No pending tasks.'}\n\n"
        f"**Passed Tasks:**\n{passed_tasks_md if passed_tasks_md else 'No passed tasks.'}\n\n"
    )
    
    return message

# Example function to send a message to the agent
def agent_send_message(message):
    human_message = HumanMessage(content=message)
    response = agent_executor.invoke(
        {"messages": [human_message]},
        config={"configurable": {"thread_id": "default", "recursion_limit": 1000}},
    )
    return response

# Function to extract the most recent message content and tool calls from the agent's response
def get_most_recent_ai_message_content_and_tool_calls(response):
    messages = response.get('messages', [])
    most_recent_content = None
    tool_calls = []

    for message in reversed(messages):
        if isinstance(message, HumanMessage):
            break
        if isinstance(message, AIMessage):
            if message.content:
                most_recent_content = message.content
            if 'tool_calls' in message.additional_kwargs:
                tool_calls.extend(message.additional_kwargs.get('tool_calls', []))

    return most_recent_content, tool_calls

@bot.event
async def on_ready():
    global tasklist_id
    print(f'Logged in as {bot.user}')
    
    tasklist_id = get_tasklist_id_by_title(service, "Schule")  # Set the task list ID
    update_tasks.start()  # Start updating tasks every minute


async def clear_channel():
    for guild in bot.guilds:
        for channel in guild.text_channels:
            if channel.name == CHANNEL_NAME:
                async for msg in channel.history(limit=None):
                    await msg.delete()

@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    if message.channel.name == CHANNEL_NAME:
        content = message.content.strip().lower()

        if content.startswith('/task-history'):
            bot_message = await message.channel.send(f"### Last 10 Completed Tasks\n TODO: Implement this feature")
            await asyncio.sleep(10)
            await bot_message.delete()
            await message.delete()
            return

        # Pass the user message to the agent
        response = agent_send_message(message.content)
        agent_message, tool_calls = get_most_recent_ai_message_content_and_tool_calls(response)

        # Send agent response back to the Discord channel
        bot_message = await message.channel.send(f"**Agent Response:** {agent_message}")
        await asyncio.sleep(30)  # Optionally delete messages after 30 seconds
        await message.delete()
        await bot_message.delete()

@tasks.loop(minutes=1)  # Loop to update tasks every 1 minute
async def update_tasks():
    global pinned_message_id
    for guild in bot.guilds:
        for channel in guild.text_channels:
            if channel.name == CHANNEL_NAME:
                # Fetch the current pinned message
                if pinned_message_id is None:
                    async for msg in channel.history(limit=10):
                        if msg.pinned and msg.author == bot.user and msg.content.startswith('### Tasks Overview'):
                            pinned_message_id = msg.id
                            break

                # Get the latest tasks overview
                tasks_overview = display_tasks(service, tasklist_id)

                # If we have a pinned message, update it
                if pinned_message_id:
                    pinned_message = await channel.fetch_message(pinned_message_id)
                    await pinned_message.edit(content=tasks_overview)
                else:
                    # If no pinned message exists, create a new one
                    bot_message = await channel.send(tasks_overview)
                    await bot_message.pin()
                    pinned_message_id = bot_message.id


@tasks.loop(minutes=30)
async def delete_non_pinned_messages():
    for guild in bot.guilds:
        for channel in guild.text_channels:
            if channel.name == CHANNEL_NAME:
                async for msg in channel.history(limit=None):
                    if not msg.pinned and not msg.content.startswith('### Pinned Tasks'):
                        await msg.delete()

# Run the bot
nest_asyncio.apply()
async def main():
    await bot.start(TOKEN)

asyncio.run(main())


Logged in as Mycroft-Mini#8841


In [1]:
from cryptography.fernet import Fernet

# Generate a key and print it
key = Fernet.generate_key()
print(f"Generated Key: {key.decode()}")

# Save the key into a file (optional)
with open("secret.key", "wb") as key_file:
    key_file.write(key)

# Load the key (if needed)
# with open("secret.key", "rb") as key_file:
#     key = key_file.read()

# Encrypt the file
with open("client_secret.json", "rb") as file:
    file_data = file.read()

fernet = Fernet(key)
encrypted_data = fernet.encrypt(file_data)

with open("client_secret.json.encrypted", "wb") as file:
    file.write(encrypted_data)

Generated Key: _j4A9j0_3B-EJQR52AsFaPP0jxa2azg9aH_4gZShdok=


In [3]:
import urllib.parse

auth_code = "4/1AVG7fiS0gpQL_iPRF8TXTaLs_r7Zm1NuKTKtm77_NhFDBqwwn4wscGG889I"
encoded_auth_code = urllib.parse.quote(auth_code)
print(encoded_auth_code)

4/1AVG7fiS0gpQL_iPRF8TXTaLs_r7Zm1NuKTKtm77_NhFDBqwwn4wscGG889I


In [6]:
import urllib.parse
import os

# Original authorization code
auth_code = "4/1AVG7fiRrglSnqKEi6eGcISyG6cEkP48GUFZaUPJmw6PQjl4lXE1axWrQyDM"

# URL-encode the authorization code
encoded_auth_code = urllib.parse.quote(auth_code)
print(f"Encoded Authorization Code: {encoded_auth_code}")

# Construct the curl command with the encoded authorization code
curl_command = f'curl -X POST https://continued-glen-enpoi-mini-bf0bea23.koyeb.app/auth -H "Content-Type: application/json" -d \'{{"code": "{encoded_auth_code}"}}\''

# Print the curl command
print(f"Curl Command: {curl_command}")

# Execute the curl command
os.system(curl_command)

Encoded Authorization Code: 4/1AVG7fiRrglSnqKEi6eGcISyG6cEkP48GUFZaUPJmw6PQjl4lXE1axWrQyDM
Curl Command: curl -X POST https://continued-glen-enpoi-mini-bf0bea23.koyeb.app/auth -H "Content-Type: application/json" -d '{"code": "4/1AVG7fiRrglSnqKEi6eGcISyG6cEkP48GUFZaUPJmw6PQjl4lXE1axWrQyDM"}'


3